In [1]:
import os
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings, Document, VectorStoreIndex, StorageContext, load_index_from_storage, PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, FunctionTool, ToolMetadata


In [2]:
llm = Ollama(model="gemma2", request_timeout=120.0, base_url="https://workstation.tail5ac87b.ts.net", verbose=True)
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")

Settings.llm = llm
Settings.embed_model = embed_model

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [3]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [4]:
PERSIST_DIR = "lyrics_store"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("8988_Kjarkas.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [5]:
lyrics_query_engine = index.as_query_engine()
qa_template_str = """
    You are an expert in Bolivian Folk music, your task is to guide and teach the user 
    about your field. Answer the user queries only with supported data in your context.
    Your context may contain complete lyrics or parts of them in different languages, but
    your answer will always be in Spanish. 

    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, 
    answer the query with detailed source information, include direct quotes and use bullet lists in your 
    answers, in one of the bullets detail the tone/sentiment of the song.
    Notes:
    Whenever the user asks for a specific song lyrics, only retrieve the lyrics with no tone/sentiment added.
    Query: {query_str}
    Answer: 
"""
qa_template = PromptTemplate(qa_template_str)
lyrics_query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [8]:
import wikipediaapi

def get_artist_wikipedia_info(artist: str) -> str:
    """
        Useful for getting artist information from wikipedia
        Use plain text as input to the tool, this should be the name of an artist or a band.
        The output of this tool is the summary of the wikipedia page corresponding to the artist.
    """
    wiki = wikipediaapi.Wikipedia("Test Wikipedia (eduardo.laruta@gmail.com)", "es")
    page = wiki.page(artist)
    if not page.exists():
        return f"Wikipedia page for {artist} does not exist"
    # print("====")
    # print(page.sections[0])
    # print("====")
    return f"{page.summary}\n{page.sections[0]}\n{page.sections[1]}"

print(get_artist_wikipedia_info("Kjarkas"))


Los Kjarkas  es un conjunto de música folklórica originado en Capinota, Cochabamba, Bolivia. Fue fundado el 23 de junio de 1971 por los hermanos Wilson Hermosa[1]​, Gonzalo Hermosa, Castel Hermosa[2]​ y Edgar Villaroel. Alcanzaron popularidad nacional e internacional junto a los compositores: Ulises Hermosa[3]​, Élmer Hermosa, Edgar Villaroel, Edwin Castellanos, Fernando Torrico, Gastón Guardia y Guillermo Ponce.
Section: Origen del nombre (1):
El nombre significa "kjarka = nada, sin afinar". Así se sentía el grupo en sus comienzos; buscaron establecer una comparación entre un instrumento sin ajustar y la situación inicial del conjunto musical.
Subsections (0):

Section: Historia (1):
En principio Los Kjarkas surgió como una necesidad económica para sus integrantes, yendo de pueblo en pueblo tocando zamba argentina, pues la música autóctona de Bolivia aun no tenía demasiada repercusión. Sin embargo, poco a poco, diversas peñas comenzaron a hacer énfasis en ritmos más propios del país c

In [52]:
lyrics_description = """
A set of all the lyrics of the songs from the Bolivian Folk Group Los Kjarkas. 
Use plain text question as input to the tool.
MANDATORY: Pass the response to the user as is, mantaining the format, do not try to summarize when using this tool. Also, you'll need
to use as much as you can the data you've gotten from wikipedia, and also the data you had before search on wikipedia. Finally,
you have to rememeber exactly the past answers, to be able to give the new answer if information that you gave before is part of
the answer. Or in other words, be like ChatGPT (GTP-3.5)
"""

tools = [
    QueryEngineTool(
        query_engine=lyrics_query_engine,
        metadata=ToolMetadata(
            name="Kjarkas_songs_lyrics",
            description=lyrics_description,
            return_direct=False
        )
    ),
    FunctionTool.from_defaults(get_artist_wikipedia_info)
]
agent = ReActAgent.from_tools(tools, verbose=True)

from llama_index.core import PromptTemplate
agent_prompt_text = """
  You are an expert in Kjarkas Folk music, your task is to guide and teach the user 
  about your field as much as you can. To do that, you'll have to use the full context. Answer the user queries only with supported data in your context.
  Your context may contain complete lyrics or parts of them in different languages, but
  your answer will always be in Spanish. 

  Context information is below.
  ---------------------
  {context_str}
  ---------------------
  Given the context information and not prior knowledge, 
  answer the query with detailed source information, include direct quotes and use bullet lists in your 
  answers, in one of the bullets detail the tone/sentiment of the song.
  Notes:
  Whenever the user asks for a specific song lyrics, only retrieve the lyrics with no tone/sentiment added.
  Query: {query_str}
  Answer: 
"""
agent_promt_tpl = PromptTemplate(agent_prompt_text)
agent.update_prompts({"agent_worker:system_prompt": agent_promt_tpl})

In [53]:
print(agent.chat("quienes son los kjarkas?").response)

> Running step d0de38bf-f0b4-4e53-a93d-8cbc4f2903d3. Step input: quienes son los kjarkas?
Thought: The current language of the user is: spanish. I need to use a tool to help me answer the question.
Action: get_artist_wikipedia_info
Action Input: {'artist': 'Los Kjarkas'}
Observation: Los Kjarkas  es un conjunto de música folklórica originado en Capinota, Cochabamba, Bolivia. Fue fundado el 23 de junio de 1971 por los hermanos Wilson Hermosa[1]​, Gonzalo Hermosa, Castel Hermosa[2]​ y Edgar Villaroel. Alcanzaron popularidad nacional e internacional junto a los compositores: Ulises Hermosa[3]​, Élmer Hermosa, Edgar Villaroel, Edwin Castellanos, Fernando Torrico, Gastón Guardia y Guillermo Ponce.
Section: Origen del nombre (1):
El nombre significa "kjarka = nada, sin afinar". Así se sentía el grupo en sus comienzos; buscaron establecer una comparación entre un instrumento sin ajustar y la situación inicial del conjunto musical.
Subsections (0):

Section: Historia (1):
En principio Los Kjar

In [54]:
print(agent.chat("cual es su canción mas conocida?").response)

> Running step 39d88a65-c085-4138-9c13-c653a6224d20. Step input: cual es su canción mas conocida?
Thought: I need to use the Kjarkas_songs_lyrics tool to find out which of their songs is most well-known.
Action: Kjarkas_songs_lyrics
Action Input: {'input': 'cuál es la canción más conocida de Los Kjarkas?'}
Observation: De acuerdo a las canciones que se proveen en el contexto,  la canción más conocida de Los Kjarkas es **"Mi Gran Amor"**. 

* En esta canción se expresa con gran sentimiento el amor y la devoción hacia una persona amada.
    * El tono/sentimiento general es romántico y apasionado.



> Running step 0ef41e0d-7933-4be0-a1d5-af5aaf115782. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Su canción más conocida es "Mi Gran Amor".
Su canción más conocida es "Mi Gran Amor".


In [55]:
print(agent.chat("dame toda la letra completa de esta canción").response)

> Running step bbd85d27-66ec-4b01-807f-a4554bdb787d. Step input: dame toda la letra completa de esta canción
Thought: I need to use the Kjarkas_songs_lyrics tool to get the lyrics for Mi Gran Amor.
Action: Kjarkas_songs_lyrics
Action Input: {'input': 'Mi Gran Amor'}
Observation: • "Mi Gran Amor" es una canción interpretada por el grupo Kjarkas.  

* Algunas de sus letras son: 

   >Amor, mi gran amor
   >mi mas bella ilusión
   >mejor canción
   >los ojos de mi alma
   >a quien cuide
   >la tierna ilusión
   >que alimente
   >tu eres al milagro
   >de este amor
   >y eres mi razón
   >para vivir

* El tono de la canción es uno de amor profundo y devoción.  



> Running step dcf21773-069a-4147-ac8d-24f10c623f69. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: La letra completa de "Mi Gran Amor" es larga. La herramienta proporcionó algunas líneas, pero no la letra completa.
La letra completa de "Mi Gran Amor" es larga. 